In [ ]:
import setuptools
%load_ext Cython
# As usual, a bit of setup

import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.cnn import *
from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient_array, eval_numerical_gradient
from cs231n.layers import *
from cs231n.fast_layers import *
from cs231n.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [ ]:
# Load the (preprocessed) CIFAR10 data.

data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape

In [ ]:
model = LinearCNN(conv_sets=[(8,2,2,2),(16,2,2,2),(32,2,2,2),(64,2,2,2)],
                pool_params=[(1,1,1),(1,1,1),(1,1,1),(1,1,1)],
                aff_dim=[128, 64], use_batchnorm=False, reg=2e-3, reset=True)
N = 50
X = np.random.randn(N, 3, 32, 32)
y = np.random.randint(10, size=N)

print -np.log(0.1)
loss, grads = model.loss(X, y)
print 'Initial loss (no regularization): ', loss
model.reg = 0.05
loss, grads = model.loss(X, y)
print 'Initial loss (with regularization): ', loss

In [ ]:
from cs231n.classifiers.linear_cnn import LinearCNN
num_train = 50
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

model = LinearCNN(conv_sets=[(8,2,2,2),(16,2,2,2),(32,2,2,2),(64,2,2,2)],
                pool_params=[(1,1,1),(1,1,1),(1,1,1),(1,1,1)],
                aff_dim=[128, 64], use_batchnorm=False, reg=2e-3, reset=True)

solver = Solver(model, small_data, num_epochs=20, batch_size=32,
                update_rule='adam', optim_config={'learning_rate': 3e-3,},
                verbose=True, print_every=10)
solver.train()

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(solver.loss_history, 'o')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(solver.train_acc_history, '-o')
plt.plot(solver.val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [ ]:
from cs231n.classifiers.linear_cnn import LinearCNN
num_train = -1
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

model = LinearCNN(conv_sets=[(8,2,2,2),(16,2,2,2),(32,2,2,2),(64,2,2,2)],
                pool_params=[(1,1,1),(1,1,1),(1,1,1),(1,1,1)],
                aff_dim=[128, 64], use_batchnorm=False, reg=0, reset=True)

In [ ]:
solver = Solver(model, small_data, num_epochs=100, batch_size=32,
                update_rule='adam', optim_config={'learning_rate': 1e-3,},
                verbose=True, print_every=500)
solver.train()

In [ ]:
solver = Solver(model, small_data, num_epochs=20, batch_size=32,
                update_rule='adam', optim_config={'learning_rate': 1e-4,},
                verbose=True, print_every=500)
solver.train()

In [ ]:
best_model = model
val_acc, test_acc = 0, 0
for i in xrange(10):
    j = i*100
    X_test = data['X_test'][j:j+100]
    X_val = data['X_val'][j:j+100]
    y_test = data['y_test'][j:j+100]
    y_val = data['y_val'][j:j+100]
    y_test_pred = np.argmax(best_model.loss(X_test), axis=1) 
    y_val_pred = np.argmax(best_model.loss(X_val), axis=1)
    val_acc += (y_val_pred == y_val).mean()
    test_acc += (y_test_pred == y_test).mean()
print 'Validation set accuracy: ', val_acc/10
print 'Test set accuracy: ', test_acc/10

In [ ]:
from cs231n.vis_utils import visualize_grid
w1 = model.params['W1']
grid = visualize_grid(w1.transpose(0, 2, 3, 1))
plt.imshow(grid.astype('uint8'))
plt.axis('off')
plt.gcf().set_size_inches(5, 5)
plt.show()

In [ ]:
from cs231n.vis_utils import visualize_grid
w1 = model.params['W1']
grid = visualize_grid(w1.transpose(0, 2, 3, 1))
# plt.imshow(255 - grid.astype('uint8'))
plt.imshow(grid.astype('uint8'))
plt.axis('off')
plt.gcf().set_size_inches(5, 5)
plt.show()

In [ ]:
from scipy import io
io.savemat('C:\\Users\\ZHB\\assignment2\\cs231n\\classifiers\\best_params.mat', model.params)

In [ ]:
X = data['X_train'][4]
x = X.transpose(1,2,0)
l = np.max(x)-np.min(x)
x -= np.min(x)
x *= 255/l
plt.imshow(x.astype('uint8'))
plt.axis('off')
plt.gcf().set_size_inches(5, 5)
plt.show()

In [ ]:
from cs231n.layers import *
# w1, b1 = model.params['W1'], model.params['b1']
w1 = np.random.randn(3,3,1,1)
b1 = np.zeros([3])
x = X[np.newaxis,:,:,:]
x = x.swapaxes(0,1)
conv_param = {'stride':1, 'pad': 1}
out, _ = conv_forward_vector(x, w1, b1, conv_param)
out = out.transpose(2,3,0,1)

x = np.zeros([32,32,3])
x[:,:,1] = out[1:-1,1:-1,0,0]
x = out[:,:,:,0]
l = np.max(x)-np.min(x)
x -= np.min(x)
x *= 255/l
print x.shape
plt.imshow(x.astype('uint8'))
plt.axis('off')
plt.gcf().set_size_inches(5, 5)
plt.show()